# OceanParcels workflow

***

In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import animation
from datetime import datetime, timedelta
from dateutil.parser import parse
from itertools import repeat
from salishsea_tools import viz_tools
from IPython.display import HTML

from parcels import FieldSet, Field, ParticleSet, JITParticle, Variable
from parcels import AdvectionRK4

%matplotlib inline
plt.rcParams['font.size'] = 12
plt.rcParams['animation.html'] = 'html5'

***

## Local functions and paths

In [2]:
def make_prefix(date, paths, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results/SalishSea/hindcast.201812/ddmmmyy/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    path = paths['NEMO']
    if 'date_cutoff' in paths and date >= parse(paths['date_cutoff']):
        path = paths['NEMO_cutoff']
    fn = '_'.join([f'SalishSea_1{res}', *repeat(date.strftime('%Y%m%d'), 2)])
    prefix = os.path.join(path, date.strftime('%d%b%y').lower(), fn)
    
    return prefix

In [3]:
# Define paths
paths = {
    'NEMO': '/results/SalishSea/hindcast.201812',
    'NEMO_cutoff': '/results2/SalishSea/hindcast.201812_annex',
    'out': '/data/bmoorema/results/parcels',
    'date_cutoff': '2016 Nov 21',
    'coords': '/data/bmoorema/MEOPAR/grid/coordinates_seagrid_SalishSea201702.nc',
    'mask': '/data/bmoorema/MEOPAR/grid/mesh_mask201702.nc',
}
mask = xr.open_dataset(paths['mask'])
coords = xr.open_dataset(paths['coords'], decode_times=False)
#daterange = [parse(d) for d in ['2017 Nov 10', '2017 Nov 12']]
date = datetime(2019, 1, 1)

***

## Simulation

User-defined routines

In [4]:
class Drifter(JITParticle):
    lon_prev = Variable('lon_prev', dtype=np.float32, to_write=False, initial=attrgetter('lon'))
    lat_prev = Variable('lat_prev', dtype=np.float32, to_write=False, initial=attrgetter('lat'))
    
    

In [5]:
def coastline_interaction(particle, fieldset, time):
    
    # Move beached particles to previous position
    #if fieldset.tmask[0, 0, particle.lat, particle.lon] == 0:
    (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
    if u == 0 and v == 0:
        particle.lon = particle.lon_prev
        particle.lat = particle.lat_prev
    
    # Update previous position
    particle.lon_prev = particle.lon
    particle.lat_prev = particle.lat

In [ ]:
# Extra landmask code
#filenames = {'lon': [paths['coords']], 'lat': [paths['coords']], 'data': [paths['mask']]}
#dimensions = {'lon': 'glamf', 'lat': 'gphif'}
#field_set.add_field(Field.from_netcdf(filenames, 'tmask', dimensions, interp_method='nearest'))

Run simulation

In [7]:
# Build filenames
NEMOin = {}
for vel in ['U', 'V']:
    NEMOin[vel] = [make_prefix(datetime(2019, 1, day), paths) + f'_grid_{vel}.nc' for day in range(1, 13)]

# Load forcing
filenames = {
    'U': {'lon': paths['coords'], 'lat': paths['coords'], 'data': NEMOin['U']},
    'V': {'lon': paths['coords'], 'lat': paths['coords'], 'data': NEMOin['V']},
}
variables = {'U': 'vozocrtx', 'V': 'vomecrty'}
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
field_set = FieldSet.from_nemo(filenames, variables, dimensions)

# Initialize particles
pset = ParticleSet.from_list(field_set, Drifter, lon=0.01*np.random.rand(100)-123.4, lat=0.01*np.random.rand(100)+49.1)

# Define behavior kernels
kernel = pset.Kernel(AdvectionRK4) + pset.Kernel(coastline_interaction)

# Execute simulation
outfile = os.path.join(paths['out'], 'results', 'coastline_on.nc')
pset.execute(
    kernel, runtime=timedelta(days=10), dt=timedelta(seconds=90),
    output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
)

INFO: Compiled DrifterAdvectionRK4coastline_interaction ==> /tmp/parcels-1896/7de99d07a6e4c73738993d57f06ed9e3.so
100% (691200.0 of 691200.0) |############| Elapsed Time: 0:02:16 Time:  0:02:16


***

## Results

In [10]:
%%capture
flags = ['off', 'on']
data, line = {}, {}

for flag in flags:
    data[flag] = xr.open_dataset(os.path.join(paths['out'], 'results', f'coastline_{flag}.nc'))

fig, axs = plt.subplots(1, 2, figsize=(12, 8))
for ax, flag in zip(axs, flags):
    ax.contour(coords.nav_lon, coords.nav_lat, mask.tmask[0, 0, ...], levels=[-0.01, 0.01], colors='k')
    ax.set_xlim([-124, -123])
    ax.set_ylim([48.5, 49.5])
    ax.set_title(f'Coastline Interaction: {flag}')
    viz_tools.set_aspect(ax)
    line[flag], = ax.plot([], [], 'ko')

def init():
    for flag in flags:
        line[flag].set_data([], []) 
    return line[flag],

def animate(hour):
    for flag in flags:
        line[flag].set_data(data[flag].lon[:, hour], data[flag].lat[:, hour]) 
    return line[flag],

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=192, interval=100)

In [11]:
HTML(anim.to_html5_video())